# Prototyping

In [1]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType

In [2]:
findspark.init()
spark = SparkSession.builder.appName('Prototyping').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-05-29 13:43:56,579 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Carriers File

In [3]:
df_carriers = spark.read.parquet('/airline-data/carriers.parquet')

Gemäss Kaggle sollte das File zwei Spalten (Code und Description) enthalten sowie 1491 Zeilen haben. Alle Zeilen sollten Distinct sein.

In [4]:
df_carriers.printSchema()

root
 |-- Code: string (nullable = true)
 |-- Description: string (nullable = true)



In [5]:
df_carriers.count()

1491

In [6]:
df_carriers.select('Code').distinct().count()

1491

In [7]:
df_carriers.select('Description').distinct().count()

1491

Da Count() und distinct().count() dieselben Werte liefern, können wir sicherstellen, dass keine Duplikate vorhanden sind.

In [8]:
df_carriers.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_carriers.columns]).show()

+----+-----------+
|Code|Description|
+----+-----------+
|   0|          0|
+----+-----------+



Das Carriers Parquet File ist somit einwandfrei. Das heisst es hat in den zwei Spalten keine null-Werte.

## Airlinedata File

In [9]:
df_airlines = spark.read.parquet('/airline-data/airline.parquet')

Das Airlines File sollte gemäss Kaggle 29 Spalten haben und nach unserer Überprüfung 123'534'969 Zeilen.

In [10]:
df_airlines.printSchema()

root
 |-- ActualElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CarrierDelay: string (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- LateAircraftDelay: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- NASDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- SecurityDelay: string (nullable = true)
 |-- TailNum: string (nullabl

Beim Erstellen des Parquet-Formats hatten wir das Problem, dass die Schema-Inferenz nicht auf Parquet übernommen wurde, sondern alle Spalten als "String" erstellt wurden.

In [11]:
len(df_airlines.columns)

29

In [12]:
df_airlines.count()

123534969

Die Anzahl Spalten und Zeilen stimmen: 29 und 123'534'969

In [13]:
df_airlines.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_airlines.columns]).show(truncate=False)

2022-05-29 13:44:30,454 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------+----+--------+--------+---------+-----------------+-----+--------+------+-------------+-------+------+-------+-------------+------------+----+
|ActualElapsedTime|AirTime|ArrDelay|ArrTime|CRSArrTime|CRSDepTime|CRSElapsedTime|CancellationCode|Cancelled|CarrierDelay|DayOfWeek|DayofMonth|DepDelay|DepTime|Dest|Distance|Diverted|FlightNum|LateAircraftDelay|Month|NASDelay|Origin|SecurityDelay|TailNum|TaxiIn|TaxiOut|UniqueCarrier|WeatherDelay|Year|
+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------+----+--------+--------+---------+-----------------+-----+--------+------+-------------+-------+------+-------+-------------+------------+----+
|0                |0      |0       |0      |0         |0         |0             |38955823     

Es hat 139'774 Nullwerte in der Spalte TailNum d.H. es gibt keine Angabe zur Tail Number (das Nummernschild von Flugzeugen). In der Spalte CancelationCode hat es 38'955'823 Nullwerte. Entweder sind die Flüge nicht gecancelled worden, oder es gibt für diesen Fall keinen Code.

In [14]:
df_airlines.filter(col('CancellationCode').isNull()).groupBy('Cancelled').count().show()

+---------+--------+
|Cancelled|   count|
+---------+--------+
|        0|38955823|
+---------+--------+



Diese Abfrage bestätigt unsere Vermutung, dass bei allen "Cancelled"-Flüge (Cancelled = 1) beim CancellationCode ein null steht. Somit macht das Datenset Sinn, dass wenn ein Flug nicht gecancelled wurde, somit auch NULL beim CancellationCodes steht.

In [15]:
spark.stop()

## Erläuterungen

### Hat es Überraschungen beim ersten Kontakt mit den Daten gegeben?

In fast einem Drittel der Zeilen hat es Nullwerte. Bei genauerem Hinschauen sind die aber nicht schlimm, resp. zu erwarten. siehe z.Bsp. Ausführungen oben bezüglich Cancellation-Code.

### Was haben wir aus dem prototyping gelernt?

Beim Einlesen der Daten aus .csv Files funktioniert die Schemainferenz nicht richtig. Alle Daten werden als Strings erkannt, obwohl das meiste eigentlich Integers sein müssten. Korrekterweise würden wir die Spalten zum richtigen Typ casten, damit die SQL Operationen effizienter ablaufen. Auf diesen Zusatzschritt haben wir verzeichtet.    
Ansonsten sind die Daten in Ordnung und auch die Nullwerte ergeben Sinn.